# Model Scoring with Watson Machine Learning

This notebook should be run in a Watson Studio project, using Default Python 3.6 runtime environment. It requires service credentials to access the following Cloud services:
* Watson Machine Learning

The notebook will submit requests to the German Credit Risk model.

#### Dependency Setup

In [1]:
!pip install --upgrade watson-machine-learning-client | tail -n 1

#### Configure Service Credentials

Update the two cells below with your Cloud API Key and your Watson Machine Learning service credentials.

In [2]:
WML_CREDENTIALS = {
  "apikey": "XycYiznO-hJ4nideySgm4bHOjuzaCAWuU8WNgOpWwms_",
  "iam_apikey_description": "Auto-generated for key 40fca5a1-ec33-48c6-a74e-7c8a4f06e6dd",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/6e8b529cf4cc4ab891306944b91199ec::serviceid:ServiceId-ec2d1bef-6024-434f-9801-dff7fdcb207a",
  "instance_id": "e3da0525-31af-4c9e-a2d2-3aad877dd536",
  "url": "https://us-south.ml.cloud.ibm.com"
}


#### Model Parameters

We use the same name for the spark model and the deployment to WML.

__Ensure that the two parameters match the model / deployment you have previously subscribed__

In [3]:
MODEL_NAME = "Spark German Risk Model"
DEPLOYMENT_NAME = "Spark German Risk Deployment"

#### Gather Model Information

In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)
wml_client.repository.list_models()

model_uid = None
wml_models = wml_client.repository.get_details()
for model_in in wml_models['models']['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

deployment_uid = None
deployment = None
scoring_url = None
wml_deployments = wml_client.deployments.get_details()
for deployment_in in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment_in['entity']['name']:
        deployment_uid = deployment_in['metadata']['guid']
        scoring_url = deployment_in['entity']['scoring_url']
        deployment = deployment_in
        break

if model_uid is None:
    print("No model ...")
    
if deployment_uid is None:
    print("No Model deployment...")
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))
print("Scoring URL: {}".format(scoring_url))

------------------------------------  -----------------------  ------------------------  ---------
GUID                                  NAME                     CREATED                   FRAMEWORK
0e47d7a8-6157-48b0-ac8a-71f05aa9cceb  Spark German Risk Model  2019-11-12T17:04:51.177Z  mllib-2.3
------------------------------------  -----------------------  ------------------------  ---------
Model id: 0e47d7a8-6157-48b0-ac8a-71f05aa9cceb
Deployment id: 95ab5f08-df78-4432-8f3a-d0835a875832
Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/e3da0525-31af-4c9e-a2d2-3aad877dd536/deployments/95ab5f08-df78-4432-8f3a-d0835a875832/online


### Send Requests to Model 

Update the values below with the payload values you want to submit to the model

In [5]:
import json

fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
    ["greater_200",17,"outstanding_credit","radio_tv",5181,"100_to_500","1_to_4",3,"male","none",4,"car_other",3,"stores","own",1,"skilled",1,"none","yes"],
    ["no_checking",34,"outstanding_credit","repairs",6008,"500_to_1000","greater_7",5,"male","co-applicant",3,"savings_insurance",35,"none", "free", 2, "management_self-employed", 1, "yes", "yes"],
    ["no_checking", 23, "outstanding_credit", "business", 2817, "500_to_1000", "1_to_4", 4, "male", "none", 4, "savings_insurance", 35, "none", "own", 1, "management_self-employed", 1, "yes", "yes"]
]

payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)

for val in scoring_response['values']:
    print(val[len(val) -1 ])


No Risk
Risk
No Risk


## Next steps


__Use the output from the cell above to validate your participation in the lab for a badge.__
